# Initialization of modules

In [5]:
from analyze_subroutines import Subroutine
from helper_functions import ReadWrite

import analysis.analyze_elm as ae
import analysis.analyze_ifs as ai
import analysis.analyze_namelist as an

import export_objects as eo

import importlib
import copy
import os

importlib.reload(eo)
importlib.reload(ae) 
importlib.reload(ai) 
importlib.reload(an) 


from mod_config import ELM_SRC
mod_dict, sub_dict, type_dict = {}, {}, {}
mod_dict, sub_dict, type_dict = eo.unpickle_unit_test("-352536b")
file = os.path.join(ELM_SRC, "biogeochem", "AllocationMod.F90")

In [6]:
subroutine_calls = {}
for s in sub_dict.keys():
    for calls in sub_dict[s].child_Subroutine.keys():
        subroutine_calls.setdefault(calls, []).extend(sub_dict[s].child_Subroutine[calls].subroutine_call)

# get default value of namelist variables
ae.insert_default(mod_dict)

namelist_dict = an.find_all_namelist()

# find parent if blocks
parent_ifs = ai.run(file)


# use ai.flatten(parent_ifs) to get all if_blocks
flat = ai.flatten(parent_ifs)
# Links if_blocks to namelist variables that are present in condition
# Links respective Variable object to namelist

an.generate_namelist_dict(mod_dict, namelist_dict, flat, subroutine_calls)
                                                    # use parent_ifs if only linking parent if_blocks

# use an.change_default() to change namelist variable default value
# Ex: an.change_default(parent_ifs, namelist_dict, "nu_com", "ECA")
ai.set_default(parent_ifs, namelist_dict)



In [7]:

example_subroutine = "allocation2_resolvenplimit".lower()
subroutine = sub_dict[example_subroutine]

In [8]:

def clear(node):
    for i in sub_dict:
        sub_dict[i].calltree=[]
    
# subroutine calltree 

clear(subroutine)
print(f"--- Subroutine Calltree with 'nu_com' == 'RD' ---")
subroutine.print_calltree(parent_ifs, namelist_dict)

# updates based on namelist variables
print()
clear(subroutine)
print(f"--- Subroutine Calltree with 'nu_com' == 'ECA' ---")
an.change_default(parent_ifs, namelist_dict, "nu_com", "ECA")
subroutine.print_calltree(parent_ifs, namelist_dict)

--- Subroutine Calltree with 'nu_com' == 'RD' ---
allocation2_resolvenplimit%926
|--->calc_nuptake_prof%1108
|--->calc_puptake_prof%1109
|--->nallocationrd%1308
|--->pallocationrd%1432

--- Subroutine Calltree with 'nu_com' == 'ECA' ---
allocation2_resolvenplimit%926
|--->nallocationecamic%1335
|--->pallocationecamic%1443


Validation

In [181]:

test_parent_ifs = ai.run("analysis/unitTest.F90")

test_subroutine = "test"

test_sub_dict = {
    test_subroutine: Subroutine(
        name=test_subroutine,
        calltree=[],
        file="test",
        start=-999,
        end=-999,
        lib_func=True,
    )
}
### REPLACE THIS WITH THE GENERATED CODE ###
test_sub_dict[test_subroutine].elmtype_access_by_ln = {
    'd' :
        [ReadWrite(status='w',ln=5),
         ],
    't' :
        [ReadWrite(status='w',ln=17),
         ReadWrite(status='w',ln=17),
         ],
    'p' :
        [ReadWrite(status='w',ln=5),
         ],
    'e' :
        [ReadWrite(status='w',ln=9),
         ],
    'j' :
        [ReadWrite(status='w',ln=9),
         ReadWrite(status='w',ln=13),
         ],
    'w' :
        [ReadWrite(status='w',ln=13),
         ],
    }

ai.set_default(test_parent_ifs, namelist_dict)

noif = ae.generate_noifs(test_sub_dict, "test", test_parent_ifs)

test_alive1, test_dead1 = ae.elmtypes(test_sub_dict, test_parent_ifs, "test", {}, noif)


Creating Subroutine test in test L-999--999


In [182]:
from itertools import zip_longest

print(f"{'Alive':<10}{' ':<25}{'Dead':<20}")
print("=" * 50)


for alive_item, dead_item in zip_longest(
    test_alive1.items(), test_dead1.items(), fillvalue=("", "")
):
    alive_var, alive_info = alive_item
    dead_var, dead_info = dead_item
    print(f"{alive_var:<5}{str(alive_info):<30}{dead_var:<5}{str(dead_info)}")

Alive                              Dead                
d    [w@L5]                        t    [w@L17, w@L17]
p    [w@L5]                        e    [w@L9]
t    [w@L17, w@L17]                j    [w@L9, w@L13]
                                   w    [w@L13]


In [183]:
# View if blocks
an.change_default(parent_ifs, namelist_dict, "nu_com", "RD")
for i in parent_ifs:
    if i.start == 1104:
        i.print_if_structure()
        
print("-----")

an.change_default(parent_ifs, namelist_dict, "nu_com", "ECA")
for i in parent_ifs:
    if i.start == 1104:
        i.print_if_structure()

start: 1104, end: 1131, depth: 1, relative: 1111, type: 1
condition: nu_com .eq. 'RD'
default: True
calls: [('calc_nuptake_prof', SubroutineCall(subname='allocation2_resolvenplimit', args=[bounds => bounds, num_soilc => num_soilc, filter_soilc => filter_soilc, cnstate_vars => cnstate_vars, nuptake_prof => nuptake_prof], ln=1108)), ('calc_puptake_prof', SubroutineCall(subname='allocation2_resolvenplimit', args=[bounds => bounds, num_soilc => num_soilc, filter_soilc => filter_soilc, cnstate_vars => cnstate_vars, puptake_prof => puptake_prof], ln=1109))]
else if:
->|start: 1111, end: 1131, depth: 1, relative: 1131, type: 2
->|condition: nu_com .eq. 'ECA' .or. nu_com .eq. 'MIC'
->|default: False
->|parent: 1104
->|children: 
->|->|->|start: 1113, end: 1129, depth: 2, relative: 1119, type: 1
->|->|->|condition: use_fates
->|->|->|default: False
->|->|->|else:
->|->|->|->|start: 1119, end: 1129, depth: 2, relative: 1129, type: 3
->|->|->|->|default: False
->|->|->|->|parent: 1113
->|->|->|pa

In [184]:

an.change_default(parent_ifs, namelist_dict, "nu_com", "ECA")
calls = ae.subroutineCalls(sub_dict, parent_ifs, example_subroutine)
for i in calls:
    print(i, calls[i])

nallocationecamic [SubroutineCall(subname='allocation2_resolvenplimit', args=[pci => pci, dt => dt, bd => bd, h2osoi_vol => h2osoi_vol, t_scalar => t_scalar, n_pcomp => n_pcomp, filter_pcomp => filter_pcomp, veg_rootc => veg_rootc_ptr, ft_index => ft_index_ptr, cn_scalar_runmean => cn_scalar_runmean_ptr, decompmicc => decompmicc, smin_nh4_vr => smin_nh4_vr, nu_com => nu_com, km_nh4_plant => km_nh4_ptr, vmax_nh4_plant => vmax_nh4_ptr, km_decomp_nh4 => km_decomp_nh4, potential_immob_vr => potential_immob_vr, plant_nh4demand_vr => plant_nh4demand_vr_ptr, col_plant_nh4demand_vr => col_plant_nh4demand_vr, fpi_nh4_vr => fpi_nh4_vr, actual_immob_nh4_vr => actual_immob_nh4_vr, smin_nh4_to_plant_vr => smin_nh4_to_plant_vr, smin_no3_vr => smin_no3_vr, km_no3_plant => km_no3_ptr, vmax_no3_plant => vmax_no3_ptr, km_decomp_no3 => km_decomp_no3, km_nit => km_nit, km_den => km_den, pot_f_nit_vr => pot_f_nit_vr, pot_f_denit_vr => pot_f_denit_vr, plant_no3demand_vr => plant_no3demand_vr_ptr, col_plant_

In [186]:
# some variables are hard coded
def compare(ifs, subroutine, namelist_variable, value1, value2):
    noif = ae.generate_noifs(sub_dict, subroutine, ifs)
                
    an.change_default(ifs, namelist_dict, namelist_variable, value1)
    alive1, dead_first_file = ae.elmtypes(sub_dict, ifs, subroutine,namelist_dict, noif)
    
    an.change_default(ifs, namelist_dict, namelist_variable, value2)
    alive2, dead_second_file = ae.elmtypes(sub_dict, ifs, subroutine,namelist_dict, noif)
    
    
    from helper_functions import summarize_read_write_status



    alive_first_file = summarize_read_write_status(alive1)
    alive_second_file = summarize_read_write_status(alive2)
    value1Alive, value2Alive = [], []
    
    for var in alive1: 
        for val in alive1[var]:
            string = f"{val.ln} {val.status} {var}"
            value1Alive.append(string)
            
    for var in alive2: 
        for val in alive2[var]:
            string = f"{val.ln} {val.status} {var}"

            value2Alive.append(string)
            
   
    value1Alive.sort()
    value2Alive.sort()
  
    alive1_m, alive2_m = ae.stagger_diff(value1Alive, value2Alive)
    ae.generate_html_comparison(alive1_m, alive2_m)
    
    return alive_first_file, alive_second_file, dead_first_file, dead_second_file, alive1, alive2, dead_first_file, dead_second_file

In [187]:
alive_first_file, alive_second_file, dead_first_file, dead_second_file, alive1, alive2, dead_first_file, dead_second_file  = compare(parent_ifs, example_subroutine, 'nu_com', "RD", "ECA")

HTML comparison saved to comparison.html


In [188]:
from itertools import zip_longest

print(f"{'Alive':<10}{' ':<85}{'Dead':<20}")
print("=" * 200)

# Use zip_longest to handle lists of different lengths
for alive_item, dead_item in zip_longest(
    alive1.items(), dead_first_file.items(), fillvalue=("", "")
):
    alive_var, alive_info = alive_item
    dead_var, dead_info = dead_item
    print(f"{alive_var:<35}{str(alive_info):<60}{dead_var:<35}{str(dead_info):<90}")

Alive                                                                                          Dead                
col_nf%plant_ndemand               [r@L1222, r@L1695, r@L1696]                                 bounds%begp                        [r@L1120]                                                                                 
col_pf%plant_pdemand               [r@L1223, r@L1713, r@L1714]                                 bounds%endp                        [r@L1120]                                                                                 
col_nf%col_plant_ndemand_vr        [w@L1222, r@L1328]                                          veg_nf%smin_nh4_to_plant           [w@L1124, rw@L1642, r@L1663]                                                              
col_pf%col_plant_pdemand_vr        [w@L1223, r@L1439, r@L1578]                                 veg_nf%smin_no3_to_plant           [w@L1125, rw@L1644, r@L1663]                                                              


In [ ]:
# insert stops into code
ae.insert(file, parent_ifs)